# SO(5) Richardson Gaudin

First, let's import the functions and packages we need we need

In [1]:
from solve_rg_eqs import solve_rgEqs, solve_rgEqs_2, solve_rgEqs_3, G_to_g
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Setting physical parameters. 

Note that $g$ is not $GL$, but instead
$g = \frac{G}{1-G\sum_k k}$. At $G_c = \frac{1}{\sum_k k}$, $g$ becomes singular. So, around this point I switch to using $1/g$ in the code.

Due to assumptions in the code, I currently require Nup and Ndown to be even numbers.

In [2]:
L = 8 # Very small system, just an illustration
Ne = 4
Nw = 4

N = Ne + Nw
ts = np.zeros(L)
vs = np.zeros(L)
vs[Ne//2] = 0
print(vs)

N = Ne+Nw + np.sum(vs)

dims = (L, Ne, Nw, vs)

G_final = .25*(2/(np.pi*L)) # 1.5 times critical coupling?
# G_final = -0.5
k = np.arange(1, 2*L+1, 2)*0.5*np.pi/L

[0. 0. 0. 0. 0. 0. 0. 0.]


Setting numerical parameters. If the following things don't work, try decreasing ``dg`` (which will change the other parameters as well). If you want more (or less) points in the output, decrease (increase) the value of ``skip``, which sets how often we remove artificial imaginary parts and save values to output (i.e. skip ``skip`` values of ``g`` before doing this).

In [3]:
dg = 0.01/N # step size of g. Scaling with N instead of L because increasing N increases difficulty
g0 = .1*dg # initial value of g
imk = g0 # scale of the imaginary parts added to k
imv = .1*g0 # scale of the imaginary parts used in the initial guess

skip=N # it's harder for larger N, so let's make it easy on us

Solving the Richardson-Gaudin equations.

Watch out, this could take a while. It will increment ``g`` from ``g0`` until
it has trouble converging, at which point it switches to incrementing ``1/g``.

Also, ignore things that look like error messages. 

## Wait for the evaluation to complete!
## The [*]: to the left of this should turn into a number [x]:

In [4]:
# seniorities = (ts, vs)
seniorities = (ts, vs)

output_df = solve_rgEqs_2(dims, G_final, k, dg=dg, g0=g0, imscale_k=imk,
                              imscale_v=imv, skip=skip)


Incrementing from 0.000125 to 0.1193662073189215
g = 0.0001, G/Gc = 0.0016
Bootstrapping from 4 to 8 fermions

Now using 4 fermions

Bad initial guess. Trying with noise.
g = 0.000125, er = 0.0003024402903392911
1th try at g = 0.000125
Smallest error from last set: 0.0003024402903392911
Noise ranges from -1.1585193920772403e-05 to 1.2419832155734385e-05
Error with 4 fermions: 6.391568584102449e-12

Now using 8 fermions

Bad initial guess. Trying with noise.
g = 0.000125, er = 0.00020072949306507872
1th try at g = 0.000125
Smallest error from last set: 0.00020072949306507872
Noise ranges from -2.5825003754549893e-06 to 9.052775798624041e-06
3th try at g = 0.000125
Smallest error from last set: 0.00020404128904996644
Noise ranges from -8.810922440854736e-06 to 1.1319536386574743e-05
5th try at g = 0.000125
Smallest error from last set: 0.00018502042033126143
Noise ranges from -1.0627514361305862e-05 to 2.7775761976792313e-06
7th try at g = 0.000125
Smallest error from last set: 0.0002112

KeyboardInterrupt: 

OK, that took a while. If there's a big pink error message about division by zero, ignore it for now.

If you see an error that says something about child processes, I suggest restarting the kernel and rerunning.

Let's look at the output, stored in a Pandas DataFrame.

To store it as a CSV, uncomment lines 2 and 3 below and input a filename when prompted. Line 4 is an example of how to load the data (or other CSV data) into a Pandas dataframe.

In [ ]:
print(output_df)
# filename = input('Name to save file to: ')
# output_df.to_csv(filename)
# new_df = pd.load_csv(filename)

# Using the results

## Pairons

Let's make a plot of the pairons. First, we need to get the results from the DataFrame. In output_df, the results for each pairon's real and imaginary part is stored as a row, with a name like ``Re(omega_0)``. Let's make lists of these column names:

In [ ]:
real_e_index = ['Re(e_{})'.format(i) for i in range(Ne)]
im_e_index = ['Im(e_{})'.format(i) for i in range(Ne)]
real_w_index = ['Re(omega_{})'.format(i) for i in range(Nw)]
im_w_index = ['Im(omega_{})'.format(i) for i in range(Nw)]

# Is this the thing we wanted?
print(real_e_index)
print(im_e_index)
print(real_w_index)
print(im_w_index)

Let's make a plot with labels and titles and legends!

In [ ]:
plt.figure(figsize=(12,12)) # Making a bigger figure for the plots

markers = ['1', '2', '3', '4', 'x', '+', '.', 'v', '^'] 
# using different shapes for the markers so we can see stacked results

plt.subplot(2,1,1) # Making 2 subplots, first for real parts
for i, ind in enumerate(real_e_index):
    # Plotting like before, but now assigning labels for the legend
    # Luckily, our rownames are decent labels already
    plt.scatter(output_df['G']*L, output_df[ind], label=ind, marker=markers[i]) 
# Doing the same for real parts of omega
for i, ind in enumerate(real_w_index): 
    plt.scatter(output_df['G']*L, output_df[ind], label=ind, marker=markers[i]) 

# Let's plot a horizontal line at 0
# plt.axhline(0)

# Let's make a thin, vertical, dotted/dashed, magenta line at G_c
g_c = L/np.sum(k)
plt.axvline(g_c, ls='-.', color='magenta', linewidth=.5)
    
# Making axes labels
plt.title('Real parts of the pairons for L = {}, Nup = {}, Ndown = {}'.format(L, Ne, Nw))
plt.xlabel('GL')
plt.ylabel('Real part')
plt.legend() 
    
plt.subplot(2,1,2) # Now moving to a second subplot
for i, ind in enumerate(im_e_index):
    plt.scatter(output_df['G']*L, output_df[ind], label=ind, marker=markers[i]) 
for i, ind in enumerate(im_w_index): 
    plt.scatter(output_df['G']*L, output_df[ind], label=ind, marker=markers[i]) 

# plt.axhline(0)
# plt.axvline(g_c, ls='-.', color='magenta', linewidth=.5)    
    
plt.title('Imaginary parts of the pairons for L = {}, Nup = {}, Ndown = {}'.format(L, Ne, Nw))
plt.xlabel('GL')
plt.ylabel('Imaginary part')
plt.legend()

plt.show()

We can change the limits of the axes to get a closer look at the behavior around G_c:

In [ ]:
plt.figure(figsize=(8,8))

plt.axvline(g_c, ls = '--', color='lightgray')
plt.axhline(0, ls = '--', color='lightgray')

for i, ind in enumerate(real_e_index):
    # s is the size of the marks (not sure the units)
    # Let's make them bigger!
    plt.scatter(output_df['G']*L, output_df[ind], label=ind, marker=markers[i], s=100) 
for i, ind in enumerate(real_w_index): 
    plt.scatter(output_df['G']*L, output_df[ind], label=ind, marker=markers[i], s=100) 
for i, ind in enumerate(im_e_index):
    plt.scatter(output_df['G']*L, output_df[ind], label=ind, marker=markers[i], s=100) 
for i, ind in enumerate(im_w_index): 
    plt.scatter(output_df['G']*L, output_df[ind], label=ind, marker=markers[i], s=100) 
    
plt.xlim(0.95*g_c, 1.05*g_c) # Looking within 5% of g_c
plt.ylim(-0.2, 0.2) # You might need to change these values

plt.legend()
plt.xlabel('GL')

plt.title('Real and imaginary parts around G_c')
plt.show()

## Energy
We can also plot energy (and other things). Let's get the energy and derivatives from the output DataFrame.

Again, if there is a division by zero error, ignore it for now. This occurs if we are taking the derivative of something too close to vertical.

In [ ]:
Es = output_df['energy']
Gs = output_df['G']
# Rescaling by appropriate factors of L
es = Es/L
gs = Gs*L # different than g in previous plot

print('Taking 1st derivative')
de = np.gradient(es, gs) # derivative de/dg
print('Taking 2nd derivative')
d2e = np.gradient(de, gs) # second derivative
print('Taking 3rd derivative')
d3e = np.gradient(d2e, gs) # third


Now, let's make a 2-by-2 plot with the energy and derivatives.

In [ ]:
plt.figure(figsize=(12, 12))

plt.subplot(2, 2, 1)
plt.plot(gs, es)
plt.xlabel('g')
plt.ylabel('e')
plt.axvline(g_c, ls=':', color='magenta')
plt.axvline(0.8585403710772574*g_c)

plt.subplot(2, 2, 2)
plt.scatter(gs, de)
plt.xlabel('g')
plt.ylabel('de/dg')
plt.axvline(g_c, ls=':', color='magenta')


plt.subplot(2, 2, 3)
plt.scatter(gs, d2e)
plt.xlabel('g')
plt.ylabel('d^2e/dg^2')
plt.axvline(g_c, ls=':', color='magenta')


plt.subplot(2, 2, 4)
plt.scatter(gs, d3e)
plt.xlabel('g')
plt.ylabel('d^3e/dg^3')
plt.axvline(g_c, ls=':', color='magenta')
# Behavior around G=0 is weird but this is due to truncation and inaccuracies in the solution around that point

plt.show()

## Momentum Distribution

Let's look at the momentum distribution within $5\%$ of my prediction for $G_c$.

In [ ]:
N_inds = ['N_{}'.format(i) for i in range(L)]

plt.figure(figsize=(12, 8))

for i, G in enumerate(Gs):
    if 0.95 < G*L/g_c < 1.05: # Only selecting within 2% of G_c
        plt.plot(k, output_df[N_inds].iloc[i], 
                label='g = {}'.format(np.round(G*L, 3)), # rounding the label to 3 decimal places
                 marker='o') # Putting a dot at each data point
        print(np.sum(output_df[N_inds].iloc[i]))
plt.xlabel('k')
plt.ylabel('N_k')
plt.legend()
plt.xlim(0, np.pi)
plt.show()

At precisely the critical coupling, it seems like Jorge's prediction of perfectly flat distribution is true, just not at the coupling he predicted.

To get a better sense of overall behavior, we can plot at a wider range of coupling. To make sure we don't end up with too many lines, I'll only plot once every 5 datapoints.

In [ ]:
plt.figure(figsize=(8,8))        
Gc = 1/np.sum(k)
for i, G in enumerate(Gs):
    n = np.sum(np.sum(output_df[N_inds].iloc[i]))
    if np.abs(n-N) > 10**-5:
        print('sum_k n_k')
        print(np.sum(output_df[N_inds].iloc[i]))
        print('G/Gc')
        print(G/Gc)
        print('g')
        print(output_df['g'].iloc[i])
    plt.plot(k, output_df[N_inds].iloc[i], 
             label='g = {}'.format(np.round(G*L, 3)), # rounding the label to 3 decimal places
             marker='o') # Putting a dot at each data point

plt.xlabel('k')
plt.ylabel('N_k')
plt.legend()
plt.xlim(0, np.pi)
plt.ylim(0, 4) # Limits on values of N_k = n_{k up} + n_{k down} + n_{-k up} + n_{-k down}
plt.show()

# Checking results with exact diagonalization

To use my exact-diagonalization code, you will need the [Quspin package](https://weinbe58.github.io/QuSpin/).
First, we import functions from my exact-diagonalization code and create a basis for ``Nup`` spin up and ``Ndown``
spin down fermions on a 1-d lattice with ``2L`` sites (since ``L`` counts only positive $k$):

In [ ]:
from exact_diag import ham_op_2, find_nk, form_basis, casimir_dict, iom_dict, quantum_operator
Nup = int(Ne + np.sum(vs)//2)
Ndown = int(Nw + np.sum(vs)//2)
print(Nup)
basis = form_basis(2*L, Nup, Ndown)
print('Number of states in the basis:')
print(basis.Ns)

Now, we create a Hamiltonian (as Quspin ``quantum_operator`` object) corresponding to our final coupling.
Note that if the number of states in the basis is too big, this operator will eat up a lot of memory. I would recommend not running any of this for L > 5.

In [ ]:
H = ham_op_2(L, G_final, k, basis=basis)

We can diagonalize this Hamiltonian. Let's do full diagonalization if the dimension of the Hilbert space is
less than 4000, and do sparse diagonalization to only get the 10 lowest-energy states if it's larger than that.

In [ ]:
dim_h = basis.Ns
print('Dimension of the Hilbert space:')
print(dim_h)

if dim_h > 4000:
    exact_energies, exact_states = H.eigsh(k=10, which='SA') # k is number of states to find, SA means find the smallest algabraic eigenvalues
else:
    exact_energies, exact_states = H.eigh() # Full diagonalization of a Hermitian matrix
    
# Creating a histogram of the energies
plt.hist(exact_energies, bins=dim_h//50)
plt.xlabel('Energy')
plt.ylabel('Number of states')
plt.title('Histogram of energies at G = {}'.format(G_final))
plt.show()

In [ ]:
def Z(a,b):
    return a*b/(a-b)
def ioms(dims, g, ks, es):
    if len(dims) == 3:
        L, Ne, Nw = dims
        vs = np.zeros(L)
    else:
        L, Ne, Nw, vs = dims
    R = np.zeros(L, dtype=np.complex128)
    for i, k in enumerate(ks):
        R[i] = g*np.sum((1-.5*vs[i])*Z(k, es))
        otherks = ks[np.arange(L) != i]
        othervs = vs[np.arange(L) != i]
        R[i] -= g*np.sum(((.5*vs[i]-1)*(.5*othervs-1)-1)
                           *Z(k, otherks))
        R[i] += vs[i]*.5
    return R

h = ham_op_2(L, output_df['G'].iloc[which_ind], k, basis)
e, v = h.eigh()

print(np.argmin(np.abs(e-e0)))
print(np.min(np.abs(e-e0)))
c0 = quantum_operator(casimir_dict(L, 0, 1), basis=basis)
c1 = quantum_operator(casimir_dict(L, 1, 1), basis=basis)
c2 = quantum_operator(casimir_dict(L, 2, 1), basis=basis)
cs = [c0, c1, c2]
print('Trv grvn state casimirs')
for c in cs:
    print(c.matrix_ele(v[:,0], v[:,0]))
print('Excited state casimirs')
for c in cs:
    print(c.matrix_ele(v[:,1], v[:,1]))

In [ ]:
from exact_diag import find_nk

def nu_i_op(i):
    nu_i_dict={'static': [['n|',[[1,]]]]}

for i, ei in enumerate(e):
    if np.abs(ei-e0) < 10**-12 or i == 0:
        print(i)
        n = find_nk(L, v[:,i], basis)
        plt.plot(n)
        print(n[1]+n[4])

In [ ]:
def ck(vs):
    cks = (.5*vs-1)**2+3*(1-.5*vs)-1
    return cks
nus  = np.zeros(L)
nus[Ne//2] = 2
ck(nus)

In [ ]:
i_ind = 1
for i, g in enumerate(output_df['g']):
    Ri = quantum_operator(iom_dict(L, g, k, k1=i_ind, mult=1.), basis=basis)
    ioms_g = ioms(dims, g, k, output_df[real_e_index].iloc[i])
    ri = ioms_g[i_ind]
    e, v = Ri.eigh()
    print(min(abs(e-ri)))
    print(np.argmin(abs(e-ri)))
    print('')

Now let's plot the energies at various couplings.

In [ ]:
e_es = []
divisible_Gs = []
for i, G in enumerate(Gs):
    if i%4 == 0: # Only using indices that are multiples of 4
        h = ham_op_2(L, G, k, basis=basis)
        e, v = h.eigsh(k=1, which = 'SA') # we only need the 0th energy
        e_es += [e[0]]
    
        divisible_Gs += [G]

plt.plot(np.array(divisible_Gs)*L, e_es, marker='o')
plt.plot(output_df['G']*L, output_df['energy'])
plt.xlabel('GL')
plt.ylabel('Energy difference')
# plt.ylim(-10**-12, 10**-12)
plt.show()

## Charge gap/chemical potential
Let's define some more quantities: First, chemical potential is (more or less) the energy
required to add a fermion to the system. We can estimate it as
\begin{equation}
\mu = \frac{E(N+1)- E(N) + E(N) - E(N-1)}{2}
= \frac{E(N+1) -E(N-1)}{2}
\end{equation}
where $E(M)$ is the ground state energy with $M$ fermions.

We can define a similar quantity, the charge gap (which I will call $\Delta$):
\begin{equation}
\Delta  = \frac{E(N+1) + E(N-1) - 2 E(N)}{2}.
\end{equation}

Let's see how these behave around our critical point:

In [ ]:
Gss = np.linspace(0, G_final, 20)

b_plus = form_basis(2*L, Nup + 1, Ndown)
b_minus = form_basis(2*L, Nup - 1, Ndown)
e_plus = np.zeros(len(Gss))
energies = np.zeros(len(Gss))
e_minus = np.zeros(len(Gss))
for i, G in enumerate(Gss):
    h_plus = ham_op_2(L, G, k, basis=b_plus)
    h = ham_op_2(L, G, k, basis=basis)
    h_minus = ham_op_2(L, G, k, basis=b_minus)
    ep, vp = h_plus.eigsh(k=1, which='SA')
    e, v = h.eigsh(k=1, which='SA')
    em, vm = h_minus.eigsh(k=1, which='SA')
    e_plus[i] = ep[0]
    energies[i] = e[0]
    e_minus[i] = em[0]

In [ ]:
mu = (e_plus - e_minus)/2
cg = (e_plus + e_minus - 2*energies)/2
plt.figure(figsize=(8,8))
plt.axhline(0, color='gray')
plt.axvline(g_c, color='gray')
plt.plot(Gss*L, mu/L, label = '$\mu/L$', marker = '+')
plt.plot(Gss*L, cg/L, label = '$\Delta/L$', marker = 'o')
plt.xlabel('GL')
plt.title('Chemical potential and charge gap')
plt.legend()
plt.show()